In [125]:
import numpy as np
from sklearn.datasets import load_iris
import typing

In [126]:
data = load_iris()

In [127]:
X, y = data["data"], data["target"]

In [129]:
train = np.append(X, y[:, None], axis = 1)

In [130]:
np.random.shuffle(train)

In [131]:
y = train[:, -1].astype(np.int8)

In [132]:
X = train[:, :4]

In [134]:
X = (X - X.min()) / (X.max() - X.min())

In [136]:
class SoftmaxRegressor():

    def __init__(self, lr: float) -> None:

        self.learning_rate = lr

    def fit(self, x: np.ndarray, y: np.ndarray, epochs: int, batch_size: int, early_stopping: int = 0):
        
        # One-hot encode targets
        y_onehot = np.eye(np.max(y) + 1)[y] # NOTE: It's possible to select the same row various times via indexing

        # Apparently numpy uses shapes starting from the last dimension (y, x) instead of (x, y)
        self.n_classes = y_onehot.shape[1]
        self.n_features = x.shape[1]
        self.coeffs = np.random.uniform(0, 1, (self.n_features, self.n_classes))
        self.biases = np.random.uniform(0, 1, self.n_classes)

        epoch_losses = np.array([])

        for epoch in range(epochs):

            batch_losses = np.array([])

            for step in range(len(x) // batch_size):

                # Set up variables for step
                batch = slice(step*batch_size, min((step + 1)*batch_size, len(x)))
                samples = x[batch, :]
                targets = y_onehot[batch, :]

                preds, logits, _ = self.predict(samples, steps=True)
                losses = categorical_crossentropy(preds, targets)

                # Calculate the derivative of all parameters with respect to all samples in the batch
                cee_gradient = -targets / preds
                softmax_true_gradient = (targets - preds) * preds
                sigmoid_gradient = logits * (np.ones(self.n_classes) - logits)
                coeff_derivative = np.repeat(samples[:, :, None], repeats=self.n_classes, axis=2)

                biases_gradients = sigmoid_gradient * softmax_true_gradient * cee_gradient[targets == 1, None]
                coeffs_gradients = coeff_derivative * biases_gradients[:, None, :]

                dbiases = np.mean(biases_gradients, axis=0)
                dcoeffs = np.mean(coeffs_gradients, axis=0)

                self.coeffs -= dcoeffs * self.learning_rate
                self.biases -= dbiases * self.learning_rate

                batch_losses = np.append(batch_losses, np.mean(losses))

            accuracy = np.sum(np.argmax(self.predict(x), axis=1) == y) / len(y)
            epoch_losses = np.append(epoch_losses, np.mean(batch_losses))
            print(f"Epoch: #{epoch} - Loss: {epoch_losses[-1]} - Acc: {accuracy:.2f}")

            if 0 < early_stopping < len(epoch_losses) - np.argmin(epoch_losses):
                break

    def predict(self, x: np.ndarray, steps: bool = False) -> typing.Union[tuple, np.ndarray]:
        coeff_out = x @ self.coeffs + self.biases
        logits = sigmoid(coeff_out)
        preds = softmax(logits)
        return (preds, logits, coeff_out) if steps else preds


def softmax(x):
    """
    Softmax function.

    :param x: 2D input matrix
    :type x: np.ndarray
    :return out: softmax of x
    :rtype: np.ndarray
    """
    x_exp = np.exp(x - np.max(x, axis=1, keepdims=True))
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)


def sigmoid(x: np.ndarray) -> np.ndarray:
    """
    Sigmoid function.

    :param x: 2D input matrix
    :type x: np.ndarray
    :return out: sigmoid of x
    :rtype: np.ndarray
    """
    return 1 / (1 + np.exp(-x))


def categorical_crossentropy(y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
    """
    Categorical cross-entropy loss function.

    :param y_pred: predicted prob dist 2D matrix
    :type y_pred: np.ndarray
    :param y_true: true prob dist 2D matrix
    :type y_true: np.ndarray
    :return out: categorical cross-entropy between y_true and y_pred
    :rtype: np.ndarray
    """
    loss = np.sum(-np.log(y_pred) * y_true, axis=1)
    return loss

model = SoftmaxRegressor(0.5)
model.fit(X, y, 1000, 150, 10)

Epoch: #0 - Loss: 1.092320196401541 - Acc: 0.33
Epoch: #1 - Loss: 1.092106169796155 - Acc: 0.33
Epoch: #2 - Loss: 1.0918923440020103 - Acc: 0.33
Epoch: #3 - Loss: 1.091678693621717 - Acc: 0.33
Epoch: #4 - Loss: 1.0914651934986819 - Acc: 0.33
Epoch: #5 - Loss: 1.0912518187055806 - Acc: 0.33
Epoch: #6 - Loss: 1.0910385445332993 - Acc: 0.33
Epoch: #7 - Loss: 1.0908253464803346 - Acc: 0.33
Epoch: #8 - Loss: 1.09061220024264 - Acc: 0.33
Epoch: #9 - Loss: 1.0903990817039144 - Acc: 0.33
Epoch: #10 - Loss: 1.0901859669263194 - Acc: 0.33
Epoch: #11 - Loss: 1.0899728321416247 - Acc: 0.33
Epoch: #12 - Loss: 1.0897596537427672 - Acc: 0.33
Epoch: #13 - Loss: 1.089546408275827 - Acc: 0.33
Epoch: #14 - Loss: 1.0893330724324095 - Acc: 0.33
Epoch: #15 - Loss: 1.0891196230424296 - Acc: 0.33
Epoch: #16 - Loss: 1.0889060370672996 - Acc: 0.33
Epoch: #17 - Loss: 1.088692291593511 - Acc: 0.34
Epoch: #18 - Loss: 1.0884783638266144 - Acc: 0.34
Epoch: #19 - Loss: 1.088264231085586 - Acc: 0.34
Epoch: #20 - Loss:

Epoch: #118 - Loss: 1.0628357056338942 - Acc: 0.79
Epoch: #119 - Loss: 1.062515103136545 - Acc: 0.76
Epoch: #120 - Loss: 1.0621931977518446 - Acc: 0.75
Epoch: #121 - Loss: 1.0618700040969016 - Acc: 0.75
Epoch: #122 - Loss: 1.0615455373759386 - Acc: 0.74
Epoch: #123 - Loss: 1.0612198133663697 - Acc: 0.74
Epoch: #124 - Loss: 1.060892848404116 - Acc: 0.74
Epoch: #125 - Loss: 1.0605646593681823 - Acc: 0.74
Epoch: #126 - Loss: 1.0602352636645331 - Acc: 0.73
Epoch: #127 - Loss: 1.0599046792092994 - Acc: 0.73
Epoch: #128 - Loss: 1.0595729244113499 - Acc: 0.73
Epoch: #129 - Loss: 1.0592400181542723 - Acc: 0.72
Epoch: #130 - Loss: 1.0589059797777949 - Acc: 0.72
Epoch: #131 - Loss: 1.0585708290586997 - Acc: 0.71
Epoch: #132 - Loss: 1.0582345861912656 - Acc: 0.71
Epoch: #133 - Loss: 1.0578972717672845 - Acc: 0.71
Epoch: #134 - Loss: 1.0575589067557039 - Acc: 0.71
Epoch: #135 - Loss: 1.0572195124819328 - Acc: 0.70
Epoch: #136 - Loss: 1.0568791106068678 - Acc: 0.69
Epoch: #137 - Loss: 1.05653772310

In [137]:
X[None, 40, :]

array([[0.91025641, 0.3974359 , 0.75641026, 0.21794872]])

In [138]:
model.predict(X[None, 40, :], False)

array([[0.19564644, 0.39559385, 0.40875971]])

In [139]:
coeffs = np.ones((4, 3))
bias = np.ones(3)
samples = np.array([[3, 7.1, 2, 5.1], [1.442, 1.2, 4, 0.11]])
targets = np.array([[0, 1, 0], [1, 0, 0]])
n_classes = 3
n_samples = 2

In [140]:
def predict(x: np.ndarray, steps: bool = False) -> typing.Union[tuple, np.ndarray]:
    coeff_out = x @ coeffs + bias
    logits = sigmoid(coeff_out)
    preds = softmax(logits)
    return (preds, logits, coeff_out) if steps else preds

preds, logits, _ = predict(samples, steps=True)
losses = categorical_crossentropy(preds, targets)

# cee_gradient = -targets / preds
# softmax_jacobian = (np.eye(n_classes) - preds.reshape((n_samples, 1, n_classes))) * preds.reshape((n_samples, n_classes, 1))
# sigmoid_gradient = logits * (np.ones(n_classes) - logits)
# coeff_derivative = samples

cee_gradient = -targets / preds
softmax_true_gradient = (targets - preds) * preds
sigmoid_gradient = logits * (np.ones(n_classes) - logits)
coeff_derivative = np.repeat(samples[:, :, None], repeats=n_classes, axis=2)

biases_gradients = sigmoid_gradient * softmax_true_gradient * cee_gradient[targets == 1, None]
coeffs_gradients = coeff_derivative * biases_gradients[:, None, :]

dbiases = np.mean(biases_gradients, axis=0)
dcoeffs = np.mean(coeffs_gradients, axis=0)

In [141]:
# Derive 3rd coeff from 1st class for the 2nd sample
# coeff_derivative[1, 2] * sigmoid_gradient[1, 0] * softmax_jacobian[1, 0, 0] * cee_gradient[1, 0]
# All coeffs in the 1st class from the 2nd sample
# coeff_derivative[1, :] * sigmoid_gradient[1, 2] * softmax_jacobian[1, 0, 0] * cee_gradient[1, 0]
# All coeffs in the 3rd class from the 2nd sample
# coeff_derivative[1, :] * sigmoid_gradient[1, 0] * softmax_jacobian[1, 0, 2] * cee_gradient[1, 0]
# All coefs in all classes from the 2nd sample (sliced softmax jacobian)
# (coeff_derivative[1, :] * coeffs.T).T * sigmoid_gradient[1, :] * softmax_jacobian[1, 0, :] * cee_gradient[1, 0]
# All coefs in all classes from the 2nd sample (full softmax jacobian and full CEE gradient)
# (coeff_derivative[1, :] * coeffs.T).T * sigmoid_gradient[1, :] * np.dot(softmax_jacobian[1, :, :], cee_gradient[1, :])
# All coefs in all classes from the 2nd sample (optimized softmax gradient)
# (coeff_derivative[1, :] * coeffs.T).T * sigmoid_gradient[1, :] * softmax_true_gradient[1, :] * cee_gradient[1, 0]
# All coeffs in all classes from all samples (method 1)
# np.repeat(coeff_derivative[:, :, None], repeats=3, axis=2) * sigmoid_gradient[:, None, :] * softmax_true_gradient[:, None, :] * cee_gradient[[0, 1], [1, 0], None, None]
# Bias in all classes from all samples
# sigmoid_gradient * softmax_true_gradient * cee_gradient[[0, 1], [1, 0], None]
# All coeffs in all classes from all samples (method 2)
# np.repeat(samples[:, :, None], repeats=3, axis=2) * (sigmoid_gradient * softmax_true_gradient * cee_gradient[targets == 1, None])[:, None, :]
# Bias in all classes from the 2nd sample
# sigmoid_gradient[1, :] * softmax_true_gradient[1, :] * cee_gradient[1, 0]